In [1]:
!git clone "https://github.com/bharatb010699/Earthquake-Forecasting.git"

Cloning into 'Earthquake-Forecasting'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 44 (delta 16), reused 23 (delta 8), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [7]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [4]:
# Some functions to help out with
def plot_predictions(test,predicted):
    plt.plot(test, color='red',label='Real Magnitude')
    plt.plot(predicted, color='blue',label='Predicted Magnitude')
    plt.title('Earthquake Magnitude Prediction')
    plt.xlabel('Time')
    plt.ylabel('Magnitude')
    plt.legend()
    plt.show()

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

In [45]:
# load dataset
dataframe = pandas.read_csv("/content/Earthquake-Forecasting/final_data.csv", header=None)
dataset = dataframe.values
X = dataset[1:,0:5].astype(float)
Y = dataset[1:,3].astype(float)
X = X[0:5000, :]
Y = Y[0:5000]
X = X[::-1]
Y = Y[::-1]
X = np.delete(X,3,1)

In [46]:
for i in range (len(Y)):
  if Y[i] < 3.5:
    Y[i] = 0
  elif Y[i] >= 3.5 and Y[i] < 4.5:
    Y[i] = 1
  elif Y[i] >= 4.5 and Y[i] < 6:
    Y[i] = 2
  else:
    Y[i] = 3 

In [47]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [53]:
def baseline_model():
  # create model
  model = Sequential()
  model.add(Dense(60, input_dim = 4, activation='relu'))
  model.add(Dense(60, input_dim = 60, activation='relu'))
  model.add(Dense(4, activation = 'softmax'))
  # Compile model
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

In [54]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=20, verbose=0)

In [55]:
kfold = KFold(n_splits=10, shuffle=True)

In [56]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 61.22% (1.84%)


In [57]:
def baseline_model_1():
  # create model
  model = Sequential()
  model.add(Dense(20, input_dim = 4, activation='relu'))
  model.add(Dense(4, activation = 'softmax'))
  # Compile model
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

In [61]:
estimator = KerasClassifier(build_fn=baseline_model_1, epochs=200, batch_size=20, verbose=0)

In [62]:
kfold = KFold(n_splits=10, shuffle=True)

In [63]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 53.32% (11.20%)
